# Simple RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG Q&A application. 

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

# What is Retrieval Augmented Generation (RAG)?

##### Definition of RAG:
Retrieval Augmented Generation (RAG) is a technique that enhances Large Language Models (LLMs) and Large Multimodal Models (LMMs) by retrieving and incorporating relevant context from a database of documents, which can include text and images, into their prompts.

##### Purpose and Utility: 
RAG aims to provide up-to-date and specific context to LMMs that are "frozen in time" due to infrequent updates from their developers, which is caused by the high costs and resource demands of training. This technique allows for more tailored and accurate responses to queries by using fresh, external data.

##### Challenges of Using LLMs Directly:
Employing LLMs without RAG can lead to several issues:
* Outdated Information: Since LLMs are trained on data up to a certain cutoff point, they may not have access to the latest developments or information, leading to outdated or inaccurate responses.
* Generic Responses: LLMs often generate generalized answers that might not fit specific use cases or queries without the additional context that RAG provides.
* Resource Intensity: Continuously retraining LLMs to stay current is resource-intensive, making it impractical for most applications, whereas RAG offers a more sustainable method to keep the model's outputs relevant.

# How RAG Works

<img src="image-742.jpeg" />

#### Summary:
With RAG, you can provide documents that:
* Contain your own data;
* Are relevant to a query, and;
* Represent the most up to date information you have.

# RAG Q&A Application
Let's look at a rather complicated looking visual representation of a basic RAG Q&A application.

<img src="https://i.imgur.com/PvlaIUO.png" />

### Imports and Utility 

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [36]:
!pip install -q -U numpy matplotlib plotly pandas scipy scikit-learn openai python-dotenv PyPDF2

In [37]:
from helpers.text_utils import TextFileLoader, CharacterTextSplitter
from helpers.vectordatabase import VectorDatabase
import asyncio

In [38]:
import nest_asyncio
nest_asyncio.apply()

# Documents

We'll be concerning ourselves with this part of the flow in the following section:

#### Chunking 
* Chunking is the process of breaking down large amounts of text into smaller segments, or chunks, that are then vectorized and stored in the database. Chunking is only required when the source documents are too large for the maximum input size imposed by models.

* For example, if the source text is a set of books, a chunking method could split the text into chapters, paragraphs, sentences, or even words, into chunks.

* Fixed-size chunking is a common chunking technique because it is simple and effective. For example, you might split an article into a set of chunks of 100 words each, or a set of 200 characters each

<img src="https://i.imgur.com/jTm9gjk.png" width="75%" />

### Loading Source Documents

So, first things first, we need some documents to work with. 

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format. 

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.


In [39]:
# TextFileLoader can load txt and pdf files
# text_loader = TextFileLoader("data/KingLear.pdf")
text_loader = TextFileLoader("data/sdm-report-final-022024-en.pdf")   
documents = text_loader.load_documents()
len(documents)

81

In [12]:
print(type(documents))
print(documents[0][:100])

<class 'list'>
1
2024 STATE OF DESIGN & MAKE
20 24STATE OF
DESIGN
&MAKE 
Insights from industry leaders on how digi


### Splitting Text Into Chunks

As we can see, there is one document - and it's the entire text of King Lear

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM. 

#### Context Window

The context window is the maximum amount of previous text the model can “see” when calculating its prediction, and is measured in tokens.

Larger context windows allow a model to use more user-provided data, like a PDF, and output longer responses, which can lead to more accurate reasoning. However, some longer context models have a “lost in the middle” issue, where content in the middle of the context window isn’t paid enough attention, which can be an issue when trying to reason with complex documents. 

#### Chunking

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length. 

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following: 
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

Let's take a peek visually at what we're doing here - and why it might be useful:

<img src="https://i.imgur.com/rtM6Ci6.png" />

As you can see (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [44]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

179

Let's take a look at some of the documents we've managed to split.

In [43]:
split_documents[0:1]

['1\n2024 STATE OF DESIGN & MAKE\n20 24STATE OF\nDESIGN\n&MAKE \nInsights from industry leaders on how digital \ntransformation is driving business resilience, \nsustainability, and talent management\n']

### Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format. 

Loosely, this means turning the text into numbers or vectors. 

Vectors represent information like words, documents, images as lists of numbers. This allows them to be easily analyzed mathematically.
[Understanding Vector Databases and Large Language Models](https://medium.com/daniel-parente/understanding-vector-databases-and-large-language-models-8758b843b12d)

There are plenty of resources that talk about this process in great detail - I'll leave this [blog](https://txt.cohere.com/sentence-word-embeddings/) from Cohere.AI as a resource if you want to deep dive a bit. 

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

While this is all baked into 1 call - let's look at some of the code that powers this process:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model. 

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

❓ Though 1536 is the default dimension of `text-embedding-3-small`, you can pass a dimension parameter and shorten the vector - what method does OpenAI use to achieve this shortening?

In models like text-embedding-3-small, "dimension" refers to the number of features used to represent each word or piece of text. To make these vectors smaller while retaining their usefulness, OpenAI likely employs a method called dimensionality reduction. This technique compresses the information in the embedding vectors, akin to summarizing a big book into a short paragraph while preserving the main ideas. By passing a smaller dimension parameter, users can instruct the model to create shorter but still meaningful embedding vectors, saving memory and speeding up calculations while retaining the essence of the text.

In [15]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [45]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

❓ What advantage does an async implementation provide for us?
>
> Answer: It provides concurrenct, scalability and improved responsiveness.
>

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus. 

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [46]:
#vector_db.search_by_text("Your servant Kent. Where is your servant Caius?", k=3)
vector_db.search_by_text("Sustainability becomes a key priority", k=3)

[('1\n2024 STATE OF DESIGN & MAKE\n20 24STATE OF\nDESIGN\n&MAKE \nInsights from industry leaders on how digital \ntransformation is driving business resilience, \nsustainability, and talent management\n',
  1.0),
 ('2\n2024  STATE OF DESIGN & MAKE\nAbout the study\nIntroduction\nKey themes  \nTop insights\nBusiness resilience\nKey insights  \nRegion distinctions  \nIndustry distinctions\nTalent\nKey insights  \nRegion distinctions  \nIndustry distinctionsSustainability\nKey insights  \nRegion distinctions  \nIndustry distinctions\nConclusion\nAppendix\nMethodology  \nGlossary  \nThank youContents\n03 60\n75\n7604\n08\n39',
  1.0),
 ('3\n2024  STATE OF DESIGN & MAKE\nThe State of Design & Make  report is a global, annual study for \nleaders who design and make places, objects, and experiences. \nIt identifies the most pressing issues shaping today’s businesses \nand helps leaders make informed, strategic decisions about how to \nprioritize and invest in the future. \nThe industries that

# Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work. 

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)): 

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this: 

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `helpers` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts. 

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-3.5-turbo"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓QUESTION:

How could we ensure our LLM always responded the same way to our inputs?

> Answer:  
> 1. Provide few shot prompts usng RAG content via the system, assistant role in messages input
> 2. Using a prompt template for consistent input formatting
> 3. Preserve the conversation state between interactions with the model
> 4. Cache responses from the model
> 5. Periodically re-evaluate the model, fine-tune as necessary 

### Creating and Prompting OpenAI's `gpt-3.5-turbo`!

Let's tie all these together and use it to prompt `gpt-3.5-turbo`!

In [51]:
from helpers.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from helpers.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way. Provide code snippets. Always provide a summary at the end. "
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
    system_role_prompt.create_message(expertise="Python"),
]

response = chat_openai.run(messages)

In [52]:
print(response)

The best way to write a loop depends on the specific use case, but a common and efficient method is to use a `for` loop in Python. Here is an example:

```python
# Example of a for loop
for i in range(5):
    print(i)
```

This loop will iterate through the numbers 0 to 4 and print each number. You can customize the range and the operations inside the loop to suit your needs.

Another option is to use a `while` loop if you need more control over the loop condition. Here is an example:

```python
# Example of a while loop
i = 0
while i < 5:
    print(i)
    i += 1
```

In this loop, the code inside the loop will continue to execute as long as the condition `i < 5` is true. The `i += 1` statement increments the value of `i` in each iteration.

Summary: The best way to write a loop is to use a `for` loop for iterating over a known sequence of elements or a `while` loop for more dynamic conditions. Make sure to choose the loop type that best fits your specific requirements.


### Retrieval Augmented Question Answering Prompt

Now we can create a RAG QA prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [20]:
RAQA_PROMPT_TEMPLATE = """ \
Use the provided context to answer the user's query. 

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".
"""

raqa_prompt = SystemRolePrompt(RAQA_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """ \
Context:
{context}

User Query:
{user_query}
"""


user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = raqa_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)
        
        return self.llm.run([formatted_system_prompt, formatted_user_prompt])

#### ❓QUESTION:

What changes could you make that would encourage the LLM to have a more well thought out and verbose response?

What is this method called?

> Answer:
> 1. Update the prmopt template with "Let's think step by step."
> 2. This is called Chain of thought prompting

In [53]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [54]:
retrieval_augmented_qa_pipeline.run_pipeline("What is this document about and who created it?")

'The document is about the "State of Design & Make" report, which is a global, annual study for leaders in industries involved in designing and creating places, objects, and experiences. The report aims to identify pressing issues impacting businesses in these industries and help leaders make informed decisions for prioritizing and investing in the future. The study was conducted by Autodesk and involved surveying and interviewing 5,399 industry leaders, futurists, and experts in architecture, engineering, construction, design and manufacturing, and media and entertainment.'

In [26]:
# retrieval_augmented_qa_pipeline.run_pipeline("Who is King Lear?")
retrieval_augmented_qa_pipeline.run_pipeline("What are the top AECO Industry skills of the future?")

'The top skills for the Architecture, Engineering, Construction, and Operations (AECO) industry of the future have not been explicitly mentioned in the text provided. However, based on the context of the report "The State of Design & Make," it emphasizes the importance of digital transformation, business resilience, sustainability, talent management, and human collaboration in industries like AECO. Therefore, skills related to digital tools and technology, adaptability to changing environments, sustainability practices, effective teamwork, and talent management may be crucial for the AECO industry\'s future success.'

In [27]:
retrieval_augmented_qa_pipeline.run_pipeline("summarize insight 5")

'Insight 5 from the 2024 State of Design & Make report focuses on achieving sustainable outcomes. It highlights the importance of prioritizing sustainability in industries that design and make places, objects, and experiences. The insight delves into strategies for achieving sustainable outcomes, including the challenges and opportunities related to sustainable practices in architecture, engineering, construction, manufacturing, and media and entertainment. Additionally, the report provides region-specific distinctions and industry-specific insights related to sustainability. Overall, this insight emphasizes the crucial role of sustainability in shaping the future of businesses in the design and manufacturing sectors.'

In [34]:
retrieval_augmented_qa_pipeline.run_pipeline("What does digital maturity  mean for D&M companies and how many companies surveyed though that they were digitally mature?")

'The concept of digital maturity for Design and Manufacturing (D&M) companies refers to their level of advancement and effectiveness in leveraging digital technologies to meet business objectives, enhance processes, and stay competitive. In the report mentioned, it discusses how digital transformation impacts business resilience, sustainability, and talent management within the industries surveyed.\n\nRegarding the number of companies surveyed that considered themselves digitally mature, this specific information is not provided in the text you provided. The report mentions that Autodesk surveyed and interviewed 5,399 industry leaders, futurists, and experts from various industries, including D&M. However, the exact number of companies that self-identified as digitally mature is not explicitly stated in the content.'

In [32]:
retrieval_augmented_qa_pipeline.run_pipeline("What does the report say about AI as trusted tool vs. AI as threat?")

"The provided context does not mention specific details about the report's findings regarding AI as a trusted tool versus AI as a threat."

### Visibility Tooling

This is great, but what if we wanted to add some visibility to our pipeline?

Let's use Weights and Biases as a visibility tool!

The first thing we'll need to do is create a Weights and Biases account and get an API key. 

You can follow the process outlined [here](https://docs.wandb.ai/quickstart) to do exactly that!

Now we can get the Weights and Biases dependency and add our key to our env. to begin!

In [ ]:
!pip install -q -U wandb

In [ ]:
wandb_key = getpass("Weights and Biases API Key: ")
os.environ["WANDB_API_KEY"] = wandb_key

In [ ]:
import wandb

os.environ["WANDB_NOTEBOOK_NAME"] = "RAQA Assignment.ipynb"
wandb.init(project="Visibility Example")

Now we can integrate Weights and Biases into our `RetrievalAugmentedQAPipeline`.

```python
if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage
                },
                inputs= {"system_prompt" : formatted_system_prompt, "user_prompt" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")
```

The main things to consider here are how to populate the various fields to make sure we're tracking useful information. 

We'll use the `text_only` flag to ensure we can get detailed information about our LLM call!

You can check out all the parameters for Weights and Biases `Trace` [here](https://github.com/wandb/wandb/blob/653015a014281f45770aaf43627f64d9c4f04a32/wandb/sdk/data_types/trace_tree.py#L166)

In [ ]:
import datetime
from wandb.sdk.data_types.trace_tree import Trace

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, wandb_project = None) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.wandb_project = wandb_project

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = raqa_prompt.create_message()

        formatted_user_prompt = user_prompt.create_message(user_query=user_query, context=context_prompt)

        
        start_time = datetime.datetime.now().timestamp() * 1000

        try:
            openai_response = self.llm.run([formatted_system_prompt, formatted_user_prompt], text_only=False)
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "success"
            status_message = (None, )
            response_text = openai_response.choices[0].message.content
            token_usage = dict(openai_response.usage)
            model = openai_response.model

        except Exception as e:
            end_time = datetime.datetime.now().timestamp() * 1000
            status = "error"
            status_message = str(e)
            response_text = ""
            token_usage = {}
            model = ""

        if self.wandb_project:
            root_span = Trace(
                name="root_span",
                kind="llm",
                status_code=status,
                status_message=status_message,
                start_time_ms=start_time,
                end_time_ms=end_time,
                metadata={
                    "token_usage" : token_usage,
                    "model_name" : model
                },
                inputs= {"system_prompt" : formatted_system_prompt, "user_prompt" : formatted_user_prompt},
                outputs= {"response" : response_text}
            )

            root_span.log(name="openai_trace")
        
        return response_text if response_text else "We ran into an error. Please try again later. Full Error Message: " + status_message

In [ ]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    wandb_project="LLM Visibility Example"
)

In [ ]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is Batman?")

In [ ]:
retrieval_augmented_qa_pipeline.run_pipeline("What happens to Cordelia?")

In [ ]:
wandb.finish()

Navigate to the Weights and Biases "run" link to see how your LLM is performing!

```
https://wandb.ai/arindam-ganguly83/Visibility%20Example/runs/y5a4ks6x?workspace=user-arindam-ganguly83
```

#### ❓QUESTION:

What is the `model_name` from the WandB `root_span` trace?

> Answer: gpt-3.5-turbo-0125

# Conclusion

In this notebook, we've gone through the steps required to create your own simple RAQA application!

Please feel free to extend this as much as you'd like. 